# Further questions:
- Are the Kc and Er values in Jerry Mullison's email from phase II or phase I OS?
- Where does the speed of sound c in the denominator go comparing 1998 TRDI field service note vs. Mullison 2017? What c should I use for the denominator, 1478.1 m/s like for calculating the depth cells in 1998 TRDI field service note??
- So I can just use the lowest measured E per cruise to define Er? Some papers do this, cite them. Or if I know that all cruises used the same ADCP, I could find the lowest measured E out of all of those cruises. (Er should be constant for a given ADCP according to Mullison 2017.)
- What year did TRDI switch from OS to OS-II?
- If the angle from the vertical is missing, can I assume 30 degrees? This is true for all configs? What are the usual/standard thetas for each instrument type?
- What years were all these instruments available? Can I potentially use cruise year to figure what instrument was being used? Diagram of evolution of ADCP models? Or years when you were supporting them?
- Send TRDI a spreadsheet of all VM-150, VM-300 to see if they can help figure out which is BB and which is NB.
- Get mis-timed cruises from Peter Gaube.
- Is direct-reading (DR) meaning that it's moored? (can try to check website)
- If blank distance/blanking interval (or whatever it's called) is missing, can I assume it's the same as bin length (or transmit pulse length maybe)? Other specific calculation questions like this.
- WH300 had only bb or nb before some time? Are all WH broadband? If so, why does Mullison 2017 have C (25%) and C (6%) values for WH?
- VM150 and/or VM300 had only bb or nb before some time?
- How to derive formula for R? Get rid of D/4 when instrument samples at midpt of depth cell?
- To TRDI FS: no frequency associated w/ BBADCP and WHADCP nominal Kc values?

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import xarray as xr

In [3]:
%run create_JASADCP_metadata_df.ipynb

# of files w/o bandwidth: 57


In [10]:
# - How many files for each type of instrument?
df['instrument_name'].value_counts()

NB-VM-150         1088
OS-38              519
OS-75              363
WH-300             188
OS-150              26
NB-VM-300           25
NB-VM-75            14
BB-VM-150           13
OS-II-38             6
NB-DR-150            3
WH-Mariner-300       3
DCP4400A             1
WH-1200              1
OS-II-75             1
WH-Mariner-600       1
Name: instrument_name, dtype: int64

# Mean volume backscattering strength eqn **(EQUATION 1)**:  
$S_v = \color{red}{C} + \color{blue}{10log((T_x + 273.16)*R^2)} - \color{orange}{L_{DBM}} - \color{green}{P_{DBW}} + \color{purple}{2 \alpha R} + 10log(10^{k_c(E-E_r)/10} - 1)$, where:  
- C = constant combining several params specific to each instrument
- $T_x$ = temperature measured at the transducer ($^{\circ}$C)
- R = along-beam range to the measurement, taken in the last quarter of the bin for Workhorse, Long Ranger, and Quartermaster, and at midpt of the bin for other instruments
- $L_{DBM}$ = 10log(transmit pulse length, meters)
- $P_{DBW}$ = 10log(transmit power, Watts)
- $\alpha$ = absorption coefficient of the water
- $k_c$ = RSSI slope (dB/count)
- $E_r$ = noise floor (counts)
- Units = dB, referenced to [meters × 4π]-1
- From Mullison 2017, meant for newer generation ADCPs, I guess

# Mean volume backscattering strength eqn (fst-003) **(EQUATION 2)**:  
$S_v = \color{red}{10log(\frac{4.47 \times 10^{-20} K_2 K_S}{c})} + \color{blue}{10log((T_x + 273.16)*R^2)} - \color{orange}{10log(P)} - \color{green}{10log(K_1)} + \color{purple}{2 \alpha R} + 10log(10^{k_c(E-E_r)/10} - 1)$
- $K_2$ = system noise factor (dimensionless)
- $K_S$ = system constant, depends on NBADCP frequency
- c = speed of sound at the scattering layer being measured
- P = transmit pulse length
- $K_1$ = real-time power into the water (Watts) 
- All others the same as Mullison 2017 eqn
- Units = dB, referenced to [meters × 4π]-1
- From fst-003, meant for NBADCPs technically

# NBADCP-specific values and calculations (fst-003):

**$C$: constant to match eqn 1 to 2 (dB)**  
$C = 10log[\frac{4.47 x 10^{-20} K_2 K_s}{c}]$, where c = speed of sound at the scattering layer being measured

**$K_2$: system noise factor (dimensionless)**

In [31]:
# From fst-003, pg 6:
arrays = [['VM', 'VM', 'VM', 'VM', 'VM', 'DR', 'DR', 'DR', 'DR', 'DR'],
          [75, 150, 300, 600, 1200, 75, 150, 300, 600, 1200]]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names = ['NBADCP Model', 'Frequency (kHz)'])
NB_K2_values = pd.DataFrame([2.5, 4.3, 4.5, 9.1, 10.5, 2.2, 3.6, 4.2, 7.1, 8.1],index=index,columns=['K2'])
NB_K2_values

K2
NBADCP Model Frequency (kHz)      
VM           75                2.5
             150               4.3
             300               4.5
             600               9.1
             1200             10.5
DR           75                2.2
             150               3.6
             300               4.2
             600               7.1
             1200              8.1

**$K_S$: system constant, depends on NBADCP freq**

In [32]:
# From fst-003, pg 10:
Ks_values = {
    'NBADCP frequency (kHz)' : [75, 150, 300, 600, 1200],
    'Ks' : [1.09E5, 4.17E5, 7.69E5, 1.56E6, 5.65E6]
}
NB_Ks_values = pd.DataFrame(Ks_values)
NB_Ks_values

,NBADCP frequency (kHz),Ks
0,75,109000.0
1,150,417000.0
2,300,769000.0
3,600,1560000.0
4,1200,5650000.0


**$K_1$: real-time power into the water (Watts)**

$K_1 = [\frac{(V_s \times a)-b}{c}]^2 \times K_{1c}$
- $V_s$ = real-time supply voltage to NBADCP transmitter (counts, V)
- a, b, c constants shown in table below
- $K_{1c}$ = power into the water during calibration at RDI factory (Watts), see table below

In [33]:
abc_values = {
    'Model' : ['DR', 'VM (110V)', 'VM (220V)', 'SC'],
    'a' : [1.397, 1.397, 0.699, 0.170],
    'b' : [6.30, 4.27, 4.27, 5.76],
    'c' : [35.10, 37.14, 37.14, 34.24]
}
NB_abc_values = pd.DataFrame(abc_values)
NB_abc_values

,Model,a,b,c
0,DR,1.397,6.30,35.10
1,VM (110V),1.397,4.27,37.14
2,VM (220V),0.699,4.27,37.14
3,SC,0.170,5.76,34.24


**$K_{1c}$: power into the water during calibration at RDI factory (Watts)**

In [34]:
# From fst-003, pg 10:
arrays = [['VM', 'VM', 'VM', 'VM', 'VM', 'DR', 'DR', 'DR', 'DR', 'DR'],
          [75, 150, 300, 600, 1200, 75, 150, 300, 600, 1200]]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names = ['NBADCP Model', 'Frequency (kHz)'])
K1c_values = pd.DataFrame([5.5, 3.3, 3.6, 1.7, 1.2, 6.1, 3.9, 4.1, 1.9, 1.3],index=index,columns=['K1c (Watts)'])
NB_K1c_values = pd.DataFrame(K1c_values)
NB_K1c_values

K1c (Watts)
NBADCP Model Frequency (kHz)             
VM           75                       5.5
             150                      3.3
             300                      3.6
             600                      1.7
             1200                     1.2
DR           75                       6.1
             150                      3.9
             300                      4.1
             600                      1.9
             1200                     1.3

**$K_c$: conversion factor for echo intensity (dB/counts)**

- For E < 200 counts, $K_c = \frac{127.3}{T_e+271}$  
- If E is 200-230 counts, calibration of $K_c$ must be done at RDI.
- If E > 230 counts, $K_c$ can't be calibrated.  
- $T_e$ = temperature of system electronics
- "$T_e$ is used to calculate $K_c$ and $E_r$. $E_r$...is particularly sensitive to changes in $T_e$, so it is crucial to obtain an accurate record of $T_e$."  
- "For DR-NBADCPs in which both system electronics and transducer assembly are immersed, $T_x$ can be substituted for $T_e$."  
- "The ambient temp for the sm electronics will generally differ from the temp recorded at the transducer. Therefore, you must independently measure and record the temperature of the system electronics."

# OS-specific values and calculations:

In [23]:
# - From Mullison (2017)
mullison_table2 = pd.read_csv(dpath + 'typical_system_characteristics_table2_mullison2017.csv')
OS_C_PDBW_values = mullison_table2.iloc[6:9]
OS_C_PDBW_values

In [17]:
# - From Jerry Mullison emails in Jan 2020:
OS_Kc_Er_values = {
    'instrument_name' : ['OS-150','OS-75','OS-38'],
    'Kc_min' : [0.41,0.36,0.36],
    'Kc_mean' : [0.42,0.39,0.37],
    'Kc_max' : [0.44,0.42,0.37],
    'Kc_sigma' : [0.012,0.019,0.004],
    'Er_min' : [18,11,5],
    'Er_mean' : [22,19,14],
    'Er_max' : [28,25,33],
    'Er_sigma' : [2.08,2.93,4.83]
}
df_OS_Kc_Er_values = pd.DataFrame(OS_Kc_Er_values)
df_OS_Kc_Er_values

,instrument_name,Kc_min,Kc_mean,Kc_max,Kc_sigma,Er_min,Er_mean,Er_max,Er_sigma
0,OS-150,0.41,0.42,0.44,0.012,18,22,28,2.08
1,OS-75,0.36,0.39,0.42,0.019,11,19,25,2.93
2,OS-38,0.36,0.37,0.37,0.004,5,14,33,4.83


# WH-specific values and calculations:

In [26]:
# - From Mullison (2017)
WH_C_PDBW_values = mullison_table2.iloc[20:23]
WH_C_PDBW_values

,Instrument,C (25%) (dB),C (6%) (dB),P_DBW Battery (dB),P_DBW Power Supply (dB),Rayleigh Distance (m)
20,Workhorse 300,-140.87,-151.64,14.0,17.5,0.87
21,Workhorse 600,-139.09,-149.14,9.0,12.5,1.75
22,Workhorse 1200,-129.44,-139.57,4.8,8.3,1.71


In [ ]:
# - From TRDI field service emails in Jan 2020:
# (they didn't specify a frequency)
WH_Kc_value = 0.42 # dB/count

# BBADCP-specific values and calculations:

In [27]:
# - From TRDI field service emails in Jan 2020:
# (they didn't specify a frequency)
BB_Kc_value = 0.45 # dB/count

# Calculations common to all ADCP:

**$R$: slant range to depth cell (m)**

$R = \frac{B + |(P-D)/2| + (N \times D) + (D/4)}{cos(\theta)} \times \frac{c'}{1475.1}$ **[method 1]**
- B = blank beyond transmit (m) (same as blanking interval?)
- P = transmit pulse length (m)
- D = depth cell length (m)
- N = depth cell number of the scattering layer being measured
- c' = wted avg of sound speed btwn transducer and depth cell (m/s)
- $\theta$ = angle of transducer beams from vertical (degrees)
- NBADCP, BBADCP, and WH sample echo intensity in the last quarter of each depth cell, not the center --> D/4 accts for this
- All other instruments sample at midpt of bin --> remove D/4, I think?
- NBADCP assumes the speed of sound is 1475.1 m/s in dtmning depth cell size; c'/1475.1 m/s corrects for variations from this value
- From fst-003, used in Lee et al. (2004) and Dwinovantyo et al. (2019)

$R = \frac{B + (L+D)/2 + ((N-1) \times D) + (D/4)}{cos(\theta)} \times \frac{c'}{c_1}$ **[method 2]**
- B = blank beyond transmit (m) (same as blanking interval?)
- D = depth cell length (m)
- N = depth cell number of the scattering layer being measured
- c' = wted avg of sound speed btwn transducer and depth cell (m/s)
- $c_1$ = sound speed used by the instrument (m/s)
- $\theta$ = angle of transducer beams from vertical (degrees)
- From Deines (1999)

$R = \frac{B + (L + D + L_a)/2 + D/4 + (n-1)D}{cos(\theta)}$ **[method 3]**
- B = blank length (m)
- D = depth cell length (m)
- n = depth cell number of the scattering layer being measured
- L = transmit pulse length
- $L_a$ = transmit lag distance (m) 
- $\theta$ = angle of transducer beams from vertical (degrees)
- This expression differs from Deines (1999), where the transmit lag distance is not included
- From Gostiaux and van Haren (2010)

**$2 \alpha R$: total absorption (dB)**

$2 \alpha R = \frac{2 \alpha_p B}{cos(\theta)} + \sum_{n=1}^{b} \alpha_n$
- b = range cell number
- $\alpha_n = \frac{2 \alpha D}{cos(\theta)}$
- $\alpha$ = absorption coefficient at that depth
- $\alpha_p$ = absorption at the profiler
- B = blank length (m)
- D = depth cell length (m)
- n = depth cell number of the scattering layer being measured
- $\theta$ = angle of transducer beams from vertical (degrees)
- From Deines (1999)

**$E_r$: real-time reference level for echo intensity (counts)**  

If $E_r$ isn't specified, use the minimum echo intensity in all of the received data, as in the following papers:
- Lee et al., 2004: "In the present paper, $E_r$ of each beam was chosen as the minimum echo intensity in all of the received data."
- Gostiaux and van Haren, 2010: "The term $E_{noise}$ is the tail value of the minimum RSSI profile over the total record."

**c: speed of sound (m/s)**  

$C = 1448.96 + 4.591T - 0.05304T^2 + 2.374 \times 10^{-4}T^3 + 1.34(S-35) + 0.0163z + 1.675 \times 10^{-7}z^2 - 0.01025T(S-35) - 7.139 \times 10^{-13}Tz^3$ **[method 1]**
- T = temp in degC
- S = salinity in PSU, parts per thousand
- z = depth in meters  
- Valid for temps from -2-30degC, salinities from 25-40 ppt, depths from 0-8000m
- Std error of the predicted SS is 0.07 m/s
- Above 1000m, $Tz^3$ term is very small and may be ignored
- From Mackenzie (1981) from Simmonds and MacLennan (2008)

$C = 1449.2 + 4.6T - 0.055T^2 + 0.00029T^3 + (1.34-0.01T)(S-35) + 0.016D$ **[method 2]**
- T = temp in degC
- S = salinity in PSU, parts per thousand
- D = depth in meters  
- From Urick (1983) from "ADCP Principles of Operation: A Practical Primer"

$C = 1492.9 + 3(T-10) - 0.006(T-10)^2 - 0.04(T-18)^2 + 1.2(S-35) - 0.01(T-18)(S-35) + D/61$
- T = temp in degC
- S = salinity in PSU, parts per thousand
- D = depth in meters  
- Predicts c correctly to within 0.1 m/s for temps from -2-23degC, salinities from 30-40ppt, depths above 500m
- Less accurate, but simpler formula good enough for many fishery applications
- From Leroy (1969) from Simmonds and MacLennan (2008)

**$\alpha$: absorption coefficient (dB/km)**

$\alpha = \frac{A_1 P_1 f_1 f^2}{f^2 + f_1^2} + \frac{A_2 P_2 f_2 f^2}{f^2 + f_2^2} + A_3 P_3 f^2$  
$A_1 = (8.86/c) 10^{0.78pH-5}$  
$f_1 = 2.8(S/35)^{0.5} 10^{[4-1245/(T+273)]}$  
$P_1 = 1$  
$A_2 = 21.44(S/c)(1+0.025T)$  
$P_2 = 1-1.37 \times 10^{-4}z + 6.2 \times 10^{-9}z^2$  
$f_2 = 8.17 \times 10^{[8-1990/(T+273)]} /[1 + 0.0018(S-35)]$  
$P_3 = 1-3.83 \times 10^{-5}z + 4.9 \times 10^{-10}z^2$  
$A_3 = \begin{cases}
  4.937 \times 10^{-4} - 2.59 \times 10^{-5}T + 9.11 \times 10^{-7}T^2 - 1.5 \times 10^{-8}T^3, & \text{if } T \le 20^{\circ}C, \\
  3.964 \times 10^{-4} - 1.146 \times 10^{-5}T + 1.45 \times 10^{-7}T^2 - 6.5 \times 10^{-10}T^3, & \text{if } T \gt 20^{\circ}C.
\end{cases}$
- f = frequency in kHz
- T = temp in $^{\circ}$C
- S = salinity in PSU, parts per thousand
- z = depth in meters  
- pH = pH (unitless)
- c = sound speed in m/s
- Claimed to predict $\alpha$ to an accuracy of 5% for temps from -1.8-30$^{\circ}$C, salinities from 30-35ppt, frequencies from 400Hz-1MHz
- pH of sw 7.8-8.2. Reasonable to assume pH=8.
- From Francois and Garrison (1982) from Simmonds and MacLennan (2008)

# For each ncfile, save the following variables along with Sv:
- Tx (temp at transducer, dims = time, units = degC)
- R (range, NBADCP samples in the last quarter of each depth cell/OS at midpt, dims = depth, units = m)
- Kc (RSSI slope, constant, units = dB/count)
- E (RSSI, dims = depth x time, units = counts)
- Er (noise floor, constant, units = counts) 
- P (transmit pulse length, 10log(P) = LDBM, constant, units = m)
- K1 (assumed or measured transmit power, 10log(K1) = PDBW, dims = time, units = Watts)
- alpha (attenuation coefficient)
- c (speed of sound, dims = depth x time, units = m/s)
- theta (angle of the transducer beams to vertical, constant, units = degrees)
- D (depth cell length, constant, units = m)
- Ks (if applicable - only for NBADCP?)
- K2 (if applicable - only for NBADCP?)
- K1c (if applicable - only for NBADCP?)
- B (blank beyond transmit (same as blanking interval?), constant, units = m)

Metadata should include:
- Instrument type
- Instrument SN
- Transmit frequency

### Narrow df down to Pacific region, OS instruments for now

In [ ]:
print(df['geo_region'].isnull().sum())
print(len(df))
dfnow = df.dropna(subset=['geo_region'])
print(len(dfnow))
dfnow = dfnow[dfnow['geo_region'].str.contains('acific')]
print(len(dfnow))
dfnow = dfnow[dfnow['instrument_name'].str.contains('OS')]
print(len(dfnow))
dfnow = dfnow[(dfnow['instrument_name']!='OS-II-38')&(dfnow['instrument_name']!='OS-II-75')] 

## Load table of C, PDBW values

In [ ]:
cpdbw = pd.read_csv(dpath + 'typical_system_characteristics_table2_mullison2017.csv')

In [ ]:
cpdbw.head()

## Load table of kc, Er values

#### Nominal Kc values from TRDI field service email:
BBADCP: 0.45 dB/count  
WHADCP: 0.42 dB/count  
OS-II-75, SN1508: 0.373 (beam 1), 0.386 (beam 2), 0.388 (beam 3), 0.384 (beam 4) = 0.38275 (avg)  
OS-II-75, SN10656: 0.398 (beam 1), 0.389 (beam 2), 0.3988 (beam 3), 0.395 (beam 4) = 0.395 (avg)  
#### Nominal Kc values from Jerry Mullison email:
Format: min-max, avg, stdev  
OS150: 0.41-0.44, 0.42, 0.012   
OS75: 0.36-0.42, 0.39, 0.019  
OS38: 0.36-0.37, 0.37, 0.004  

#### Nominal Er values from Jerry Mullison email:
Format: min-max, avg, stdev  
OS150: 18-28, 22, 2.08  
OS75: 11-25, 19, 2.93  
OS38: 5-33, 14, 4.83  

In [ ]:
# - Calc avg Kc for OS-II w/ SNs
print(np.mean([0.373, 0.386, 0.388, 0.384]))
print(np.mean([0.398, 0.389, 0.398, 0.395]))

## Load WOD data (T,S)

## Compute Sv

In [11]:
ncfile = '/opt/acoustic-variability/data/JASADCP/ncfiles/00216_short.nc'
ncnow = xr.open_dataset(ncfile)
strnow = ncnow.attrs['cruise_sonar_summary']
#re.findall('arro', strnow, re.IGNORECASE)
ncnow.attrs['cruise_sonar_summary'].split('\n')

[' #DATA_DATES: 1992/05/01 16:35:00 --- to --- 1992/05/26 06:19:00',
 ' #LON_RANGE: 112.67 W --- to ---  71.50 W',
 ' #LAT_RANGE:  33.04 S --- to ---  27.90 S',
 ' #DEPTH_RANGE:     12 --- to --- 484 m',
 ' #SAC_CRUISE_ID: 00216 ',
 ' #PLATFORM_NAME: R/V Knorr',
 ' #PRINCIPAL_INVESTIGATOR_NAME: M.Kosro',
 ' #PI_INSTITUTION: Oregon State University ',
 ' #PI_COUNTRY:  USA',
 ' #PROJECT: WOCE (One-time line)',
 ' #CRUISE_NAME: ship_tag=kn9205 woce_tag=P06E EXPOCODE=316N138_3',
 ' #PORTS: Valpariso to Easter Island',
 ' #GEOGRAPHIC_REGION: SE Pacific                                  ',
 ' #PROCESSED_BY:  Oregon State University',
 ' #NAVIGATION: GPS                          ',
 ' #QUALITY_NAV: good ',
 ' #GENERAL_INFORMATION: ',
 'CRUISE NOTES',
 '  CHIEF SCIENTIST ON SHIP     : H.Bryden',
 '    INSTITUTE                 : James Rennell Centre for Ocean Research',
 '    COUNTRY                   : UK ',
 '  SIGNIFICANT DATA GAPS       : some short gaps',
 '  SPECIAL SHIP TRACK PATTERNS :'

In [ ]:
dfnow.iloc[0]

Mean volume backscattering strength:  
$S_v = C + 10log((T_x + 273.16)*R^2) - L_{DBM} - P_{DBW} + 2 \alpha R + 10log(10^{k_c(E-E_r)/10} - 1)$

In [ ]:
ncfile = dpath + 'JASADCP/ncfiles/00001_short.nc'
nc = xr.open_dataset(ncfile)

In [ ]:
cols = df.columns.values

In [ ]:
cols[0:-1]

In [ ]:
nc

In [ ]:
nc['depth']

In [ ]:
nc['amp']

In [ ]:
nc['amp'].plot()

In [ ]:
nc['depth'].values

In [ ]:
#nct['ADCP_CONFIG'].attrs['model_name']
nct['ADCP_CONFIG'].attrs

# TESTING/OLD

In [ ]:
fnames = sorted(os.listdir(dpath + 'JASADCP/ncfiles'))

In [ ]:
nc_counter = len(fnames)
geo_region = [None]*nc_counter

In [ ]:
ifile = 0
for fname in fnames:
    ncfile = dpath + 'JASADCP/ncfiles/' + fname
    ncnow = xr.open_dataset(ncfile)
    strnow = ncnow.attrs['cruise_sonar_summary']
    geo_regionnow = re.findall("GEOGRAPHIC_REGION *: *((?:\S+ )*\S+)", strnow)
    if geo_regionnow:
        geo_region[ifile] = geo_regionnow[0]
    ifile = ifile+1

In [ ]:
df = pd.concat(
    [pd.Series(fnames,name='fname'), pd.Series(geo_region,name='geo_region')],
    axis=1)